In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 5
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = [128]  # LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
    outputSize = 128  # 从高维映射到低维的神经元个数
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 200)
train label shape: (20000, 1)
eval data shape: (5000, 200)


In [5]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [6]:
"""
构建模型，模型的架构如下：
1，利用Bi-LSTM获得上下文的信息
2，将Bi-LSTM获得的隐层输出和词向量拼接[fwOutput;wordEmbedding;bwOutput]
3，将2所得的词表示映射到低维
4，hidden_size上每个位置的值都取时间步上最大的值，类似于max-pool
5，softmax分类
"""

class RCNN(object):
    """
    RCNN 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            
        # 定义两层双向LSTM的模型结构

        with tf.name_scope("Bi-LSTM"):
            fwHiddenLayers = []
            bwHiddenLayers = []
            for idx, hiddenSize in enumerate(config.model.hiddenSizes):

                with tf.name_scope("Bi-LSTM-" + str(idx)):
                    # 定义前向LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)
                    # 定义反向LSTM结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)

                fwHiddenLayers.append(lstmFwCell)
                bwHiddenLayers.append(lstmBwCell)

            # 实现多层的LSTM结构， state_is_tuple=True，则状态会以元祖的形式组合(h, c)，否则列向拼接
            fwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=fwHiddenLayers, state_is_tuple=True)
            bwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=bwHiddenLayers, state_is_tuple=True)

            # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
            # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
            # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
            outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(fwMultiLstm, bwMultiLstm, self.embeddedWords, dtype=tf.float32)
            fwOutput, bwOutput = outputs
            
        with tf.name_scope("context"):
            shape = [tf.shape(fwOutput)[0], 1, tf.shape(fwOutput)[2]]
            self.contextLeft = tf.concat([tf.zeros(shape), fwOutput[:, :-1]], axis=1, name="contextLeft")
            self.contextRight = tf.concat([bwOutput[:, 1:], tf.zeros(shape)], axis=1, name="contextRight")
            
        # 将前向，后向的输出和最早的词向量拼接在一起得到最终的词表征
        with tf.name_scope("wordRepresentation"):
            self.wordRepre = tf.concat([self.contextLeft, self.embeddedWords, self.contextRight], axis=2)
            wordSize = config.model.hiddenSizes[-1] * 2 + config.model.embeddingSize 
        
        with tf.name_scope("textRepresentation"):
            outputSize = config.model.outputSize
            textW = tf.Variable(tf.random_uniform([wordSize, outputSize], -1.0, 1.0), name="W2")
            textB = tf.Variable(tf.constant(0.1, shape=[outputSize]), name="b2")
            
            # tf.einsum可以指定维度的消除运算
            self.textRepre = tf.tanh(tf.einsum('aij,jk->aik', self.wordRepre, textW) + textB)
            
        # 做max-pool的操作，将时间步的维度消失
        output = tf.reduce_max(self.textRepre, axis=1)
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss

In [7]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [8]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = RCNN(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
        builder = tf.saved_model.builder.SavedModelBuilder("../model/Bi-LSTM/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
                if currentStep % config.training.checkpointEvery == 0:
                    # 保存模型的另一种方法，保存checkpoint文件
                    path = saver.save(sess, "../model/Bi-LSTM/model/my-model", global_step=currentStep)
                    print("Saved model checkpoint to {}\n".format(path))
                    
        inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
                  "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

        outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

        prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
                                                                                      method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
        legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
        builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
                                            signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

        builder.save()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel:0/grad/hist is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel:0/grad/sparsity is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias:0/grad/hist is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias:0/grad/sparsity is illegal; using bidirectional_rnn/fw/multi_rnn_cell/cell_0/lstm_cell/bias_0

2018-12-28T16:15:04.971202, step: 47, loss: 0.6929523944854736, acc: 0.4922, auc: 0.5355, precision: 0.0, recall: 0.0
2018-12-28T16:15:05.383545, step: 48, loss: 0.6917513608932495, acc: 0.4922, auc: 0.5342, precision: 0.0, recall: 0.0
2018-12-28T16:15:05.767563, step: 49, loss: 0.6976913213729858, acc: 0.5625, auc: 0.4441, precision: 0.0, recall: 0.0
2018-12-28T16:15:06.175974, step: 50, loss: 0.6926724314689636, acc: 0.4844, auc: 0.513, precision: 0.0, recall: 0.0
2018-12-28T16:15:06.564988, step: 51, loss: 0.699394941329956, acc: 0.5703, auc: 0.5416, precision: 0.0, recall: 0.0
2018-12-28T16:15:06.956706, step: 52, loss: 0.6993716955184937, acc: 0.5703, auc: 0.5239, precision: 0.0, recall: 0.0
2018-12-28T16:15:07.362967, step: 53, loss: 0.6948812007904053, acc: 0.5156, auc: 0.4768, precision: 0.0, recall: 0.0
2018-12-28T16:15:07.744026, step: 54, loss: 0.6936355829238892, acc: 0.5, auc: 0.571, precision: 0.0, recall: 0.0
2018-12-28T16:15:08.124826, step: 55, loss: 0.6956313848495483

2018-12-28T16:15:48.020024, step: 116, loss: 0.6930278539657593, acc: 0.4844, auc: 0.4635, precision: 0.0, recall: 0.0
2018-12-28T16:15:48.424290, step: 117, loss: 0.6860237121582031, acc: 0.4375, auc: 0.4901, precision: 0.0, recall: 0.0
2018-12-28T16:15:48.810438, step: 118, loss: 0.6977503895759583, acc: 0.5078, auc: 0.491, precision: 0.0, recall: 0.0
2018-12-28T16:15:49.199376, step: 119, loss: 0.6894234418869019, acc: 0.4531, auc: 0.4953, precision: 0.0, recall: 0.0
2018-12-28T16:15:49.572671, step: 120, loss: 0.6965242624282837, acc: 0.5, auc: 0.5466, precision: 0.0, recall: 0.0
2018-12-28T16:15:49.958755, step: 121, loss: 0.7061136960983276, acc: 0.5547, auc: 0.539, precision: 0.0, recall: 0.0
2018-12-28T16:15:50.357681, step: 122, loss: 0.7016212940216064, acc: 0.5312, auc: 0.5115, precision: 0.0, recall: 0.0
2018-12-28T16:15:50.748059, step: 123, loss: 0.702599048614502, acc: 0.5469, auc: 0.4772, precision: 0.0, recall: 0.0
2018-12-28T16:15:51.138703, step: 124, loss: 0.6919157

2018-12-28T16:16:15.897013, step: 185, loss: 0.6982711553573608, acc: 0.4688, auc: 0.4955, precision: 0.0, recall: 0.0
2018-12-28T16:16:16.289376, step: 186, loss: 0.6976273059844971, acc: 0.4766, auc: 0.5049, precision: 0.0, recall: 0.0
2018-12-28T16:16:16.691428, step: 187, loss: 0.6880168914794922, acc: 0.5547, auc: 0.5516, precision: 0.0, recall: 0.0
2018-12-28T16:16:17.095153, step: 188, loss: 0.7034684419631958, acc: 0.4297, auc: 0.4666, precision: 0.0, recall: 0.0
2018-12-28T16:16:17.492774, step: 189, loss: 0.6853783130645752, acc: 0.5781, auc: 0.5683, precision: 0.0, recall: 0.0
2018-12-28T16:16:17.922889, step: 190, loss: 0.693905770778656, acc: 0.5, auc: 0.516, precision: 0.0, recall: 0.0
2018-12-28T16:16:18.324669, step: 191, loss: 0.6956160664558411, acc: 0.4844, auc: 0.4949, precision: 0.0, recall: 0.0
2018-12-28T16:16:18.710665, step: 192, loss: 0.6902813911437988, acc: 0.5391, auc: 0.4778, precision: 0.0, recall: 0.0
2018-12-28T16:16:19.106655, step: 193, loss: 0.691922

2018-12-28T16:16:58.675464, step: 253, loss: 0.6751733422279358, acc: 0.4766, auc: 0.7487, precision: 0.0, recall: 0.0
2018-12-28T16:16:59.087036, step: 254, loss: 0.6791950464248657, acc: 0.5391, auc: 0.7696, precision: 0.0, recall: 0.0
2018-12-28T16:16:59.546517, step: 255, loss: 0.6841363310813904, acc: 0.4844, auc: 0.6068, precision: 0.0, recall: 0.0
2018-12-28T16:16:59.978120, step: 256, loss: 0.6834651827812195, acc: 0.4766, auc: 0.6565, precision: 0.0, recall: 0.0
2018-12-28T16:17:00.360912, step: 257, loss: 0.6902270913124084, acc: 0.5781, auc: 0.5766, precision: 0.0, recall: 0.0
2018-12-28T16:17:00.776083, step: 258, loss: 0.6883634924888611, acc: 0.5703, auc: 0.6682, precision: 0.0, recall: 0.0
2018-12-28T16:17:01.183456, step: 259, loss: 0.6876323223114014, acc: 0.5391, auc: 0.6008, precision: 0.0, recall: 0.0
2018-12-28T16:17:01.576849, step: 260, loss: 0.6919277310371399, acc: 0.4375, auc: 0.5851, precision: 0.0, recall: 0.0
2018-12-28T16:17:01.965977, step: 261, loss: 0.6

...

2018-12-28T16:18:51.854613, step: 454, loss: 0.4001741409301758, acc: 0.8516, auc: 0.914, precision: 0.8714, recall: 0.8592
2018-12-28T16:18:52.256085, step: 455, loss: 0.4346463084220886, acc: 0.8438, auc: 0.9155, precision: 0.7619, recall: 0.9057
2018-12-28T16:18:52.668589, step: 456, loss: 0.37355220317840576, acc: 0.8438, auc: 0.9162, precision: 0.8909, recall: 0.7778
2018-12-28T16:18:53.066840, step: 457, loss: 0.33626580238342285, acc: 0.8828, auc: 0.9392, precision: 0.9143, recall: 0.8767
2018-12-28T16:18:53.465817, step: 458, loss: 0.45421284437179565, acc: 0.8125, auc: 0.8759, precision: 0.8542, recall: 0.7069
2018-12-28T16:18:53.856984, step: 459, loss: 0.3714432716369629, acc: 0.875, auc: 0.9, precision: 0.9259, recall: 0.8065
2018-12-28T16:18:54.247873, step: 460, loss: 0.4051814675331116, acc: 0.8203, auc: 0.9027, precision: 0.8679, recall: 0.7419
2018-12-28T16:18:54.648785, step: 461, loss: 0.3981325030326843, acc: 0.8438, auc: 0.9048, precision: 0.86, recall: 0.7679
2018

2018-12-28T16:19:33.857206, step: 519, loss: 0.27170294523239136, acc: 0.9219, auc: 0.9528, precision: 0.8852, recall: 0.9474
2018-12-28T16:19:34.260189, step: 520, loss: 0.3733632266521454, acc: 0.8828, auc: 0.9045, precision: 0.8451, recall: 0.9375
2018-12-28T16:19:34.642630, step: 521, loss: 0.3312065303325653, acc: 0.8828, auc: 0.9367, precision: 0.8545, recall: 0.8704
2018-12-28T16:19:35.075667, step: 522, loss: 0.36691024899482727, acc: 0.8594, auc: 0.9276, precision: 0.8, recall: 0.9123
2018-12-28T16:19:35.485913, step: 523, loss: 0.2242797613143921, acc: 0.9375, auc: 0.9737, precision: 0.9722, recall: 0.9211
2018-12-28T16:19:35.885612, step: 524, loss: 0.2844694256782532, acc: 0.8828, auc: 0.9464, precision: 0.9194, recall: 0.8507
2018-12-28T16:19:36.282268, step: 525, loss: 0.23778952658176422, acc: 0.9141, auc: 0.9688, precision: 0.8947, recall: 0.9107
2018-12-28T16:19:36.692190, step: 526, loss: 0.2423611581325531, acc: 0.9141, auc: 0.9721, precision: 0.9545, recall: 0.8873


2018-12-28T16:20:00.522433, step: 585, loss: 0.2020949274301529, acc: 0.9297, auc: 0.9785, precision: 0.913, recall: 0.9545
2018-12-28T16:20:00.955634, step: 586, loss: 0.22120273113250732, acc: 0.9062, auc: 0.9809, precision: 0.9444, recall: 0.85
2018-12-28T16:20:01.352766, step: 587, loss: 0.33069878816604614, acc: 0.8438, auc: 0.9514, precision: 0.9216, recall: 0.746
2018-12-28T16:20:01.758477, step: 588, loss: 0.2833271026611328, acc: 0.8984, auc: 0.936, precision: 0.9194, recall: 0.8769
2018-12-28T16:20:02.137824, step: 589, loss: 0.26276513934135437, acc: 0.9141, auc: 0.9606, precision: 0.8947, recall: 0.9107
2018-12-28T16:20:02.584971, step: 590, loss: 0.2750913202762604, acc: 0.8984, auc: 0.9631, precision: 0.9032, recall: 0.8889
2018-12-28T16:20:03.032486, step: 591, loss: 0.3170105516910553, acc: 0.8438, auc: 0.9369, precision: 0.902, recall: 0.7541
2018-12-28T16:20:03.422182, step: 592, loss: 0.2994408905506134, acc: 0.8906, auc: 0.9512, precision: 0.9516, recall: 0.8429
201

2018-12-28T16:20:42.263103, step: 649, loss: 0.19746491312980652, acc: 0.9375, auc: 0.9719, precision: 0.931, recall: 0.931
2018-12-28T16:20:42.681341, step: 650, loss: 0.20845240354537964, acc: 0.9375, auc: 0.9671, precision: 0.9623, recall: 0.8947
2018-12-28T16:20:43.095950, step: 651, loss: 0.11273534595966339, acc: 0.9609, auc: 0.9985, precision: 0.9833, recall: 0.9365
2018-12-28T16:20:43.468193, step: 652, loss: 0.19911807775497437, acc: 0.8984, auc: 0.9811, precision: 0.9825, recall: 0.8235
2018-12-28T16:20:43.867848, step: 653, loss: 0.22460713982582092, acc: 0.9375, auc: 0.9422, precision: 0.9452, recall: 0.9452
2018-12-28T16:20:44.263483, step: 654, loss: 0.1606791764497757, acc: 0.9453, auc: 0.9797, precision: 0.9545, recall: 0.9403
2018-12-28T16:20:44.656203, step: 655, loss: 0.1971195489168167, acc: 0.9453, auc: 0.9613, precision: 0.9492, recall: 0.9333
2018-12-28T16:20:45.063548, step: 656, loss: 0.21022692322731018, acc: 0.9375, auc: 0.966, precision: 0.9492, recall: 0.91

2018-12-28T16:21:23.564156, step: 713, loss: 0.15329992771148682, acc: 0.9453, auc: 0.9885, precision: 0.9508, recall: 0.9355
2018-12-28T16:21:23.973165, step: 714, loss: 0.19166824221611023, acc: 0.9297, auc: 0.9799, precision: 1.0, recall: 0.8696
2018-12-28T16:21:24.416072, step: 715, loss: 0.2981071174144745, acc: 0.9062, auc: 0.9365, precision: 0.9074, recall: 0.875
2018-12-28T16:21:24.828134, step: 716, loss: 0.30151283740997314, acc: 0.8984, auc: 0.9358, precision: 0.8615, recall: 0.9333
2018-12-28T16:21:25.248619, step: 717, loss: 0.2530254125595093, acc: 0.9297, auc: 0.9572, precision: 0.8793, recall: 0.9623
2018-12-28T16:21:25.668705, step: 718, loss: 0.22168810665607452, acc: 0.9219, auc: 0.9713, precision: 0.9143, recall: 0.9412
2018-12-28T16:21:26.098812, step: 719, loss: 0.2819673717021942, acc: 0.8906, auc: 0.9394, precision: 0.9118, recall: 0.8857
2018-12-28T16:21:26.499299, step: 720, loss: 0.22787432372570038, acc: 0.9141, auc: 0.9705, precision: 0.9483, recall: 0.873
